In [1]:
import numpy as np
from myhdl import *

from src.utils import RTLblocks

In [2]:
# SFIFO test

ref = {'d': [], 'empty': True, 'full': False, 'ovflo': False, 'len': 8}
def readFrom(fifo):
    if fifo['empty'] or fifo['ovflo']:
        return None
    else:
        val = fifo['d'].pop(0)
        if len(fifo['d']) == 0:
            fifo['empty'] = True
        if fifo['full']:
            fifo['full'] = False
        return val
    
def wrTo(fifo, val):
    if fifo['full']:
        fifo['ovflo'] = True
    if len(fifo['d']) == fifo['len']-1:
        fifo['full'] = True
    if fifo['empty']:
        fifo['empty'] = False
    if not fifo['ovflo']:
        fifo['d'].append(val)
        
def rstRef():
    global ref
    ref = {'d': [], 'empty': True, 'full': False, 'ovflo': False, 'len': 8}

In [3]:
rstRef()
print(ref)
for ii in range(10):
    wrTo(ref, ii)
    print(ref)
for ii in range(3):
    val = readFrom(ref)
    print(val, ref)

{'d': [], 'empty': True, 'full': False, 'ovflo': False, 'len': 8}
{'d': [0], 'empty': False, 'full': False, 'ovflo': False, 'len': 8}
{'d': [0, 1], 'empty': False, 'full': False, 'ovflo': False, 'len': 8}
{'d': [0, 1, 2], 'empty': False, 'full': False, 'ovflo': False, 'len': 8}
{'d': [0, 1, 2, 3], 'empty': False, 'full': False, 'ovflo': False, 'len': 8}
{'d': [0, 1, 2, 3, 4], 'empty': False, 'full': False, 'ovflo': False, 'len': 8}
{'d': [0, 1, 2, 3, 4, 5], 'empty': False, 'full': False, 'ovflo': False, 'len': 8}
{'d': [0, 1, 2, 3, 4, 5, 6], 'empty': False, 'full': False, 'ovflo': False, 'len': 8}
{'d': [0, 1, 2, 3, 4, 5, 6, 7], 'empty': False, 'full': True, 'ovflo': False, 'len': 8}
{'d': [0, 1, 2, 3, 4, 5, 6, 7], 'empty': False, 'full': True, 'ovflo': True, 'len': 8}
{'d': [0, 1, 2, 3, 4, 5, 6, 7], 'empty': False, 'full': True, 'ovflo': True, 'len': 8}
None {'d': [0, 1, 2, 3, 4, 5, 6, 7], 'empty': False, 'full': True, 'ovflo': True, 'len': 8}
None {'d': [0, 1, 2, 3, 4, 5, 6, 7], 'emp

In [4]:
@block
def testbench(bw=8, autoCheck=True):
    period = 10
    i_clk = Signal(bool(0))
    i_rst = ResetSignal(0, active=bool(1), isasync=False)
    i_wren = Signal(bool(0))
    i_d = Signal(intbv(0, min=-2**(bw-1), max=2**(bw-1)))
    i_rden = Signal(bool(0))
    o_d = Signal(i_d.val)
    o_dv = Signal(bool(0))
    o_empty = Signal(bool(0))
    o_full = Signal(bool(0))
    o_ovflo = Signal(bool(0))
    DEPTH = 8
    
    uut = RTLblocks.SFIFO(i_clk, i_rst, i_wren, i_d, i_rden, o_d, o_dv, o_empty, o_full, o_ovflo, DEPTH)
    
    @always(delay(int(period//2)))
    def clk_gen():
        i_clk.next = not i_clk
        
    @instance
    def stim():
        rstRef()
        yield i_clk.negedge
        i_rst.next = bool(1)
        yield i_clk.negedge
        i_rst.next = bool(0)
        if autoCheck:
            assert o_empty.val == ref['empty'], "Empty signals mismatch"
            assert o_full.val == ref['full'], "Full signals mismatch"
            assert o_ovflo.val == ref['ovflo'], "Overflow signals mismatch"
        print('write 10 values - overflow check')
        for ii in range(10):
            val = np.random.randint(-2**(bw-1), 2**(bw-1)-1)
            i_d.next = val
            i_wren.next = bool(1)
            yield i_clk.negedge
            wrTo(ref, val)
            if autoCheck:
                assert o_empty.val == ref['empty'], f"Empty signals mismatch, ii = {ii}"
                assert o_full.val == ref['full'], f"Full signals mismatch, ii = {ii}"
                assert o_ovflo.val == ref['ovflo'], f"Overflow signals mismatch, ii = {ii}"
                if not (ref['empty'] or ref['ovflo']):
                    assert o_dv.val == True, f"Valid flag must be True, ii = {ii}"
                    assert o_d.val == ref['d'][0]
        i_wren.next = bool(0)
        print('reset fifo to continue')
        rstRef()
        i_rst.next = bool(1)
        yield i_clk.negedge
        i_rst.next = bool(0)
        print('write 5 values...')
        for ii in range(8):
            val = np.random.randint(-2**(bw-1), 2**(bw-1)-1)
            i_d.next = val
            i_wren.next = bool(1)
            yield i_clk.negedge
            wrTo(ref, val)
            if autoCheck:
                assert o_empty.val == ref['empty'], f"Empty signals mismatch, ii = {ii}"
                assert o_full.val == ref['full'], f"Full signals mismatch, ii = {ii}"
                assert o_ovflo.val == ref['ovflo'], f"Overflow signals mismatch, ii = {ii}"
                if not (ref['empty'] or ref['ovflo']):
                    assert o_dv.val == True, f"Valid flag must be True, ii = {ii}"
                    assert o_d.val == ref['d'][0]
        i_wren.next = bool(0)
        print('then read 2...')
        for ii in range(2):
            i_rden.next = bool(1)
            yield i_clk.negedge
            readFrom(ref)
            if autoCheck:
                assert o_empty.val == ref['empty'], f"Empty signals mismatch, ii = {ii}"
                assert o_full.val == ref['full'], f"Full signals mismatch, ii = {ii}"
                assert o_ovflo.val == ref['ovflo'], f"Overflow signals mismatch, ii = {ii}"
                if not (ref['empty'] or ref['ovflo']):
                    assert o_dv.val == True, f"Valid flag must be True, ii = {ii}"
                    assert o_d.val == ref['d'][0]
        i_rden.next = bool(0)
        print('write 5 more - to full state...')
        for ii in range(5):
            val = np.random.randint(-2**(bw-1), 2**(bw-1)-1)
            i_d.next = val
            i_wren.next = bool(1)
            yield i_clk.negedge
            wrTo(ref, val)
            if autoCheck:
                assert o_empty.val == ref['empty'], f"Empty signals mismatch, ii = {ii}"
                assert o_full.val == ref['full'], f"Full signals mismatch, ii = {ii}"
                assert o_ovflo.val == ref['ovflo'], f"Overflow signals mismatch, ii = {ii}"
                if not (ref['empty'] or ref['ovflo']):
                    assert o_dv.val == True, f"Valid flag must be True, ii = {ii}"
                    assert o_d.val == ref['d'][0]
        i_wren.next = bool(0)
        print('read all values finally')
        for ii in range(8):
            i_rden.next = bool(1)
            yield i_clk.negedge
            readFrom(ref)
            if autoCheck:
                assert o_empty.val == ref['empty'], f"Empty signals mismatch, ii = {ii}"
                assert o_full.val == ref['full'], f"Full signals mismatch, ii = {ii}"
                assert o_ovflo.val == ref['ovflo'], f"Overflow signals mismatch, ii = {ii}"
                if not (ref['empty'] or ref['ovflo']):
                    assert o_dv.val == True, f"Valid flag must be True, ii = {ii}"
                    assert o_d.val == ref['d'][0]            
        print(' **** Test passed **** ')
        raise StopSimulation
        
    return instances()

In [5]:
tb = testbench(bw=8, autoCheck=True)
tb.config_sim(trace=False)
tb.run_sim()

write 10 values - overflow check
reset fifo to continue
write 5 values...
then read 2...
write 5 more - to full state...
read all values finally
 **** Test passed **** 


In [6]:
bw = 8

i_clk = Signal(bool(0))
i_rst = ResetSignal(0, active=bool(1), isasync=False)
i_wren = Signal(bool(0))
i_d = Signal(intbv(0, min=-2**(bw-1), max=2**(bw-1)))
i_rden = Signal(bool(0))
o_d = Signal(i_d.val)
o_dv = Signal(bool(0))
o_empty = Signal(bool(0))
o_full = Signal(bool(0))
o_ovflo = Signal(bool(0))
DEPTH = 256
    
inst = RTLblocks.SFIFO(i_clk, i_rst, i_wren, i_d, i_rden, o_d, o_dv, o_empty, o_full, o_ovflo, DEPTH)
inst.convert(hdl='VHDL', path='./vhdl', initial_values=True);

with open('./vhdl/SFIFO.vhd') as f:
    print(f.read())

-- File: ./vhdl\SFIFO.vhd
-- Generated by MyHDL 0.11
-- Date: Thu Jun 20 12:42:33 2019


library IEEE;
use IEEE.std_logic_1164.all;
use IEEE.numeric_std.all;
use std.textio.all;

use work.pck_myhdl_011.all;

entity SFIFO is
    port (
        i_clk: in std_logic;
        i_rst: in std_logic;
        i_wren: in std_logic;
        i_d: in signed (7 downto 0);
        i_rden: in std_logic;
        o_d: out signed (7 downto 0);
        o_dv: out std_logic;
        o_empty: out std_logic;
        o_full: out std_logic;
        o_ovflo: out std_logic
    );
end entity SFIFO;
-- Synchronous (one-clock domain) FIFO (first word fall through type)
-- ~~~~ input ports ~~~~
-- :param i_clk: clock;
-- :param i_rst: synchronous reset for FIFO state;
-- :param i_wren: write enable;
-- :param i_d: writing data;
-- :param i_rden: read enable;
-- ~~~~ output ports ~~~~
-- :param o_d: read data;
-- :param o_dv: output data valid flag;
-- :param o_empty: empty flag;
-- :param o_full: full flag;
-- :param 